In [1]:
import pandas as pd
import pandas_ta as ta
import matplotlib.pyplot as plt
from backtesting import Strategy, Backtest

In [2]:
'''
This strategy takes into account RSI for entry signals and a exponential moving average for trend detection.
is intended to be used with a 15m timeframe, but can be used with any timeframe, just test it out.
The backtest is done with backtesting.py and is intended to throw the results from it, the results are going to be optimized.
'''

'\nThis strategy takes into account RSI for entry signals and a exponential moving average for trend detection.\nis intended to be used with a 15m timeframe, but can be used with any timeframe, just test it out.\nThe backtest is done with backtesting.py and is intended to throw the results from it, the results are going to be optimized.\n'

READ & CLEAN THE DATA SET

In [2]:
df = pd.read_csv('/mnt/c/Users/samue/Documents/ALGORITHMIC-TRADING-PROJECTS/backtesting strategies/data_sets/AUDUSD_Candlestick_15_M_BID_21.04.2004-16.04.2022.csv')
df.drop(columns=['Unnamed: 0'], inplace=True)
df['Date'] = pd.to_datetime(df['Date'])
df

,Date,Open,High,Low,Close,Volume
0,2004-04-21 00:00:00,0.73057,0.73215,0.73007,0.73163,36625.7781
1,2004-04-21 00:15:00,0.73118,0.73138,0.72993,0.73033,24471.4404
2,2004-04-21 00:30:00,0.73034,0.73155,0.72995,0.73072,37426.0350
3,2004-04-21 00:45:00,0.73087,0.73113,0.73016,0.73031,35051.6650
4,2004-04-21 01:00:00,0.73076,0.73105,0.72924,0.73030,28241.7052
...,...,...,...,...,...,...
445622,2022-04-15 14:45:00,0.73921,0.73925,0.73905,0.73922,254.3800
445623,2022-04-15 15:00:00,0.73922,0.73929,0.73902,0.73921,288.3600
445624,2022-04-15 15:15:00,0.73922,0.73949,0.73916,0.73921,153.3400
445625,2022-04-15 15:30:00,0.73921,0.73926,0.73900,0.73915,602.3500


CREATING THE INDICATORS

In [3]:
ema_period = 200
rsi_period = 2
atr_period = 14

df['EMA200'] = ta.ema(df.Close, length = ema_period)
df['RSI2'] = ta.rsi(df.Close, length = rsi_period)
df['ATR'] = ta.atr(df.High, df.Low, df.Close, length = atr_period)
df.dropna()

,Date,Open,High,Low,Close,Volume,EMA200,RSI2,ATR
199,2004-04-23 01:45:00,0.73350,0.73382,0.73202,0.73257,27202.6019,0.730482,18.378872,0.001180
200,2004-04-23 02:00:00,0.73244,0.73252,0.73097,0.73139,19891.3269,0.730491,8.591923,0.001210
201,2004-04-23 02:15:00,0.73142,0.73163,0.73032,0.73073,35509.4298,0.730494,5.384456,0.001217
202,2004-04-23 02:30:00,0.73091,0.73184,0.73040,0.73092,28288.6682,0.730498,22.123096,0.001233
203,2004-04-23 02:45:00,0.73099,0.73176,0.73059,0.73161,22374.8962,0.730509,65.917323,0.001228
...,...,...,...,...,...,...,...,...,...
445622,2022-04-15 14:45:00,0.73921,0.73925,0.73905,0.73922,254.3800,0.741561,70.686696,0.000239
445623,2022-04-15 15:00:00,0.73922,0.73929,0.73902,0.73921,288.3600,0.741537,64.310182,0.000242
445624,2022-04-15 15:15:00,0.73922,0.73949,0.73916,0.73921,153.3400,0.741514,64.310182,0.000248
445625,2022-04-15 15:30:00,0.73921,0.73926,0.73900,0.73915,602.3500,0.741491,20.319205,0.000249


SLICING THE DATASET

In [ ]:
# #60% of the dataset
# df60 = df[0:270000]  
# #40% of the dataset
# df40 = df[270000:449192]  
# df60.dropna()
# df40.dropna()

GETTING THE SIGNALS

In [4]:
emasignal = [0] * len(df)
backcandles = 8

for row in range(backcandles-1, len(df)):
    upt = 1
    dnt = 1
    for i in range(row-backcandles, row+1):
        if df.High[row] >= df.EMA200[row]:
            dnt = 0
        if df.Low[row] <= df.EMA200[row]:
            upt = 0
            
    if upt == 1 and dnt == 1:
        emasignal[row] = 3
    elif upt == 1:
        emasignal[row] = 2
    elif dnt == 1:
        emasignal[row] = 1
        
df['EMASignal'] = emasignal

# Considering RSI and creating a total Signal
TotalSignal = [0] * len(df)

for row in range(0, len(df)):
    TotalSignal[row] = 0
    
    if df.EMASignal[row] == 1 and df.RSI2[row] >= 90:
        TotalSignal[row] = 1
        
    if df.EMASignal[row] == 2 and df.RSI2[row] <= 10:
        TotalSignal[row] = 2
        
df['TotalSignal'] = TotalSignal
df.dropna(inplace=True)
df
    

,Date,Open,High,Low,Close,Volume,EMA200,RSI2,ATR,EMASignal,TotalSignal
199,2004-04-23 01:45:00,0.73350,0.73382,0.73202,0.73257,27202.6019,0.730482,18.378872,0.001180,2,0
200,2004-04-23 02:00:00,0.73244,0.73252,0.73097,0.73139,19891.3269,0.730491,8.591923,0.001210,2,2
201,2004-04-23 02:15:00,0.73142,0.73163,0.73032,0.73073,35509.4298,0.730494,5.384456,0.001217,0,0
202,2004-04-23 02:30:00,0.73091,0.73184,0.73040,0.73092,28288.6682,0.730498,22.123096,0.001233,0,0
203,2004-04-23 02:45:00,0.73099,0.73176,0.73059,0.73161,22374.8962,0.730509,65.917323,0.001228,2,0
...,...,...,...,...,...,...,...,...,...,...,...
445622,2022-04-15 14:45:00,0.73921,0.73925,0.73905,0.73922,254.3800,0.741561,70.686696,0.000239,1,0
445623,2022-04-15 15:00:00,0.73922,0.73929,0.73902,0.73921,288.3600,0.741537,64.310182,0.000242,1,0
445624,2022-04-15 15:15:00,0.73922,0.73949,0.73916,0.73921,153.3400,0.741514,64.310182,0.000248,1,0
445625,2022-04-15 15:30:00,0.73921,0.73926,0.73900,0.73915,602.3500,0.741491,20.319205,0.000249,1,0


In [5]:
def SIGNAL():
    return df.TotalSignal

BACKTEST USING ATR RELATED SL&TP

In [12]:
class MyStrat(Strategy):
    riskpct = 0.2
    
    def init(self):
        super().init()
        self.signal = self.I(SIGNAL)

    def next(self):
        super().next()
        slatr = 1.2*self.data.ATR[-1]
        TPSLRatio = 1.2
        price = self.data.Close[-1] 
        
        if self.position:
            self.position.close()
            
        if self.signal == 2 and len(self.trades) == 0:
            sl1 = price - slatr
            tp1 = price + slatr*TPSLRatio    # 1e-4 is 1 pip
             
            self.buy(sl=sl1, tp=tp1, size= self.riskpct)
            
        elif self.signal == 1 and len(self.trades) == 0:
            sl2 = price + slatr
            tp2 = price - slatr*TPSLRatio    # 1e-4 is 1 pip
                                  
            self.sell(sl=sl2, tp=tp2, size= self.riskpct)

bt = Backtest(df, MyStrat, cash=100, margin=1/50, commission = 0.0001, trade_on_close=False)
stat = bt.run()
print(stat)
trades_size = stat['_trades']#['Size'] 
print(trades_size) # shows the size of the trades
# #plt.bar(trades_size, height=trades_size.count())
# #plot = bt.plot(plot_volume=False)
# #print(plot)

/tmp/ipykernel_10400/969539582.py:29: UserWarning: Data index is not datetime. Assuming simple periods, but `pd.DateTimeIndex` is advised.
  bt = Backtest(df, MyStrat, cash=100, margin=1/50, commission = 0.0001, trade_on_close=False)


Start                                   199.0
End                                  445626.0
Duration                             445427.0
Exposure Time [%]                    7.367296
Equity Final [$]                     0.053127
Equity Peak [$]                    106.317805
Return [%]                         -99.946873
Buy & Hold Return [%]                0.875002
Return (Ann.) [%]                         0.0
Volatility (Ann.) [%]                     NaN
Sharpe Ratio                              NaN
Sortino Ratio                             NaN
Calmar Ratio                              0.0
Max. Drawdown [%]                   -99.95003
Avg. Drawdown [%]                   -9.892551
Max. Drawdown Duration               444331.0
Avg. Drawdown Duration                40484.0
# Trades                              17389.0
Win Rate [%]                          47.9211
Best Trade [%]                       1.135133
Worst Trade [%]                     -1.078226
Avg. Trade [%]                    

BACKTEST USING SIZE CALCULATION BY RISK% AND SL SIZE

In [8]:
# class MyStrat(Strategy):
#     riskpct = 0.02
#     contract_size = 100000
    
#     def init(self):
#         super().init()
#         self.signal1 = self.I(SIGNAL)  

#     def next(self):
#         super().next()
        
#         if self.position:
#             self.position.close()
            
#         if self.signal1 == 2 and len(self.trades) == 0:
#             o, h, l, c = self.data.Open[-1], self.data.High[-1], self.data.Low[-1], self.data.Close[-1]
#             size1 = (self.equity*self.riskpct)/((o - l)*self.contract_size)     
            
#             self.buy(sl = l - (5e-4), tp = h + (100e-4), size=(size1))
            
#         if self.signal1 == 1 and len(self.trades) == 0:
#             o, h, l, c = self.data.Open[-1], self.data.High[-1], self.data.Low[-1], self.data.Close[-1]
#             size2 = (self.equity*self.riskpct)/((h - c)*self.contract_size)
                                    
#             self.sell(sl = h + (5e-4), tp = l - (100e-4), size = (size2))

# bt = Backtest(df, MyStrat, cash=100, margin=1/50, commission = 0.00, trade_on_close=False)
# stat = bt.run()
# print(stat)
# trades_size = stat['_trades']#['Size'] 
# print(trades_size) # shows the size of the trades
# # plt.bar(trades_size, height=trades_size.count())
# # plot = bt.plot(plot_volume=False)
# # print(plot)

/tmp/ipykernel_10400/3480300794.py:27: UserWarning: Data index is not datetime. Assuming simple periods, but `pd.DateTimeIndex` is advised.
  bt = Backtest(df, MyStrat, cash=100, margin=1/50, commission = 0.00, trade_on_close=False)


AssertionError: size must be a positive fraction of equity, or a positive whole number of units

OPTIMIZATION

In [ ]:
# Since Optimization takes some time to analize the data, I already have done it and the best parameters are: EMA=200, RSI=2

# stats = bt.optimize(
#     ema_window = [50, 200],
#     rsi_window = range(2, 4),
#     maximize = 'Equity Final [$]',
# )

# print(stats)
# print(stats._strategy)
# bt.plot(plot_volume=False, plot_pl=False)

CALCULATION OF POSITION SIZE BY RISK% AND SL SIZE

In [ ]:
# c, h, o, l = self.data.Close[-1], self.data.High[-1], self.data.Open[-1], self.data.Low[-1]
# riskpct = 0.002
# contract_size = 100000
# size2 = (self.equity*riskpct)//(h - c)*contract_size
# size1 = (self.equity*riskpct)//(o - l)*contract_size

CALCULATION OF SL ON LOW AND HIGH OF SIGNAL CANDLE 

In [ ]:
# c, h, o, l = self.data.Close[-1], self.data.High[-1], self.data.Open[-1], self.data.Low[-1]
# self.sell(limit=c, sl=h+4e-4, tp=c - 2*(c-1), size=riskpct)
# self.buy(limit=c, sl=l-4e-4, tp=c + 2*(c-1), size=riskpct)

HOW TO CALCULATE MARTINGALE TRADES

In [ ]:
#if (self.signal1 > 0 and len(self.trades) == 0 and len(self.closed_trades) > 0 and self.closed_trades[-1].pl < 0):
#    self.mysize = self.mysize*2
#elif len(self.closed_trades) > 0 and self.closed_trades[-1].pl > 0:
#    self.mysize = self.initsize